#### Imports

In [37]:
# Workforce Scheduling for Anonymous Bank Call Center (Python)

# prepare for Python version 3x features and functions
from __future__ import division, print_function

# import packages for analysis and modeling
import pandas as pd  # data frame operations
import numpy as np  # arrays and math functions
import datetime
from rpy2.robjects import r  # interface from Python to R

#### Erlang C queueing theory

In [38]:
# Erlang C queueing theory  
# input c = number of servers (positive integer)
#       r = ratio of arrival rate over service rate 
# output = probability of waiting in queue (min 0, max 1)
# adapted from Pedro Canadilla (2014) function 
# C_erlang in the R queueing package 
def erlang_C (c = 1, r = 0):
    if (c <= 0):
        return(1)
    if (r <= 0):
        return(0)
    c = int(c)    
    tot = 1
    for i in range(c-1):
        i = i + 1
        tot = 1 + (tot * i * (1/r))
    return(max(0, min(1, (r * (1/tot)) / (c - (r * (1 - (1/tot)))))))


#### Get and clean data

In [39]:
# focus upon February 1999
call_center_input_data = pd.read_table('data_anonymous_bank_february.txt')

# delete PHANTOM calls
call_center_data = \
    call_center_input_data[call_center_input_data['outcome'] != 'PHANTOM']

# negative VRU times make no sense... drop these rows from data frame
call_center_data = call_center_data[call_center_data['vru_time'] >= 0]

# calculate wait time as sum of vru_time and q_time
call_center_data['wait_time'] = call_center_data['vru_time'] + call_center_data['q_time']

# define date variable with apply and lambda function
call_center_data['date'] = \
    call_center_data['date']\
    .apply(lambda d: datetime.datetime.strptime(str(d), '%y%m%d'))

# define day of week as an integer 0 = Monday 6 = Sunday
call_center_data['day_of_week'] = \
    call_center_data['date'].apply(lambda d: d.weekday())
# use dictionary object for mapping day_of_week to string
day_of_week_to_string = {0 : 'Monday', 
     1 : 'Tuesday', 
     2 : 'Wednesday', 
     3 : 'Thursday', 
     4 : 'Friday',
     5 : 'Saturday',
     6 : 'Sunday'}
call_center_data['day_of_week'] = \
    call_center_data['day_of_week'].map(day_of_week_to_string)
# check structure and contents of the data frame
call_center_data

,vru+line,call_id,customer_id,priority,type,date,vru_entry,vru_exit,vru_time,q_start,q_exit,q_time,outcome,ser_start,ser_exit,ser_time,server,wait_time,day_of_week
0,AA0101,34536,0.0,0,PS,1999-02-01,7:02:47,7:02:56,9,0:00:00,0:00:00,0,AGENT,7:02:55,7:05:41,166,DORIT,9,Monday
1,AA0101,34537,0.0,0,PS,1999-02-01,7:31:45,7:31:55,10,0:00:00,0:00:00,0,AGENT,7:31:54,7:31:59,5,NO_SERVER,10,Monday
2,AA0101,34538,0.0,0,NW,1999-02-01,7:54:38,7:54:51,13,0:00:00,0:00:00,0,AGENT,7:54:49,7:56:21,92,TOVA,13,Monday
3,AA0101,34539,23317894.0,2,PS,1999-02-01,8:09:16,8:09:22,6,8:09:22,8:09:27,5,HANG,0:00:00,0:00:00,0,NO_SERVER,11,Monday
4,AA0101,34540,48178511.0,2,PS,1999-02-01,8:19:42,8:19:47,5,8:19:47,8:20:21,34,AGENT,8:20:20,8:22:04,104,MICHAL,39,Monday
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33339,AA0216,5607,0.0,0,IN,1999-02-26,14:14:43,14:15:05,22,0:00:00,0:00:00,0,HANG,0:00:00,0:00:00,0,NO_SERVER,22,Friday
33340,AA0216,5608,0.0,0,IN,1999-02-26,14:33:45,14:33:50,5,0:00:00,0:00:00,0,HANG,0:00:00,0:00:00,0,NO_SERVER,5,Friday
33341,AA0216,5609,0.0,0,IN,1999-02-28,11:29:39,11:29:48,9,11:29:48,11:33:39,231,HANG,0:00:00,0:00:00,0,NO_SERVER,240,Sunday
33342,AA0216,5610,0.0,0,IN,1999-02-28,12:18:20,12:18:30,10,12:18:30,12:18:49,19,AGENT,12:18:48,12:22:54,246,YITZ,29,Sunday


#### Get calls with hold times > 120

In [54]:
wait_time_120 = call_center_data.query('wait_time > 120')
wait_time_120

,vru+line,call_id,customer_id,priority,type,date,vru_entry,vru_exit,vru_time,q_start,q_exit,q_time,outcome,ser_start,ser_exit,ser_time,server,wait_time,day_of_week,call_hour
15,AA0101,34551,56346398.0,1,PS,1999-02-01,1900-01-01 10:03:42,10:03:48,6,10:03:48,10:06:10,142,AGENT,10:06:09,10:08:20,131,MICHAL,148,Monday,10
22,AA0101,34558,9288283.0,2,PS,1999-02-01,1900-01-01 11:17:50,11:17:56,6,11:17:56,11:20:34,158,AGENT,11:20:33,11:22:10,97,YITZ,164,Monday,11
42,AA0101,34578,0.0,0,NW,1999-02-01,1900-01-01 15:33:09,15:33:18,9,15:33:18,15:36:07,169,AGENT,15:36:06,15:37:11,65,YITZ,178,Monday,15
44,AA0101,34580,0.0,0,NW,1999-02-01,1900-01-01 15:56:21,15:56:30,9,15:56:30,15:59:53,203,AGENT,15:59:52,16:02:33,161,AVNI,212,Monday,15
52,AA0101,34588,33966326.0,1,PS,1999-02-01,1900-01-01 18:11:02,18:11:08,6,18:11:08,18:15:31,263,AGENT,18:15:31,18:16:53,82,IDIT,269,Monday,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33328,AA0216,5596,0.0,0,IN,1999-02-24,1900-01-01 18:34:19,18:34:28,9,18:34:28,18:39:19,291,AGENT,18:39:18,18:41:22,124,BENSION,300,Wednesday,18
33334,AA0216,5602,0.0,0,IN,1999-02-25,1900-01-01 15:35:48,15:35:58,10,15:35:58,15:40:40,282,HANG,0:00:00,0:00:00,0,NO_SERVER,292,Thursday,15
33337,AA0216,5605,0.0,0,IN,1999-02-26,1900-01-01 09:52:29,9:52:38,9,9:52:38,9:56:06,208,AGENT,9:56:06,10:02:52,406,TOVA,217,Friday,9
33341,AA0216,5609,0.0,0,IN,1999-02-28,1900-01-01 11:29:39,11:29:48,9,11:29:48,11:33:39,231,HANG,0:00:00,0:00:00,0,NO_SERVER,240,Sunday,11


#### EDA

In [40]:
# examine frequency of calls by day of week
print(wait_time_120['day_of_week'].value_counts())

Tuesday      6339
Wednesday    6289
Sunday       6120
Thursday     5785
Monday       5722
Friday       1931
Saturday      857
Name: day_of_week, dtype: int64


In [55]:
# create an ordered table for Frequency of calls
table_data = wait_time_120.loc[:,['day_of_week', 'call_hour']]
day_of_week_to_ordered_day_of_week = {'Monday' : '2_Monday', 
     'Tuesday' : '3_Tuesday', 
     'Wednesday' : '4_Wednesday', 
     'Thursday' : '5_Thursday', 
     'Friday' : '6_Friday',
     'Saturday' : '7_Saturday',
     'Sunday' : '1_Sunday'}
table_data['ordered_day_of_week'] = table_data['day_of_week'].map(day_of_week_to_ordered_day_of_week)
ordered_table_data = pd.crosstab(table_data['ordered_day_of_week'], table_data['call_hour'], margins = False)
ordered_table_data

call_hour,0,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
ordered_day_of_week,,,,,,,,,,,,,,,,,,,
1_Sunday,0,0,12,61,68,96,74,45,28,52,54,56,21,3,2,0,0,9,3
2_Monday,0,0,20,27,85,68,68,28,33,27,84,35,28,46,10,27,9,26,37
3_Tuesday,0,1,13,10,71,124,99,32,47,55,126,93,63,48,17,30,43,28,14
4_Wednesday,0,0,21,66,72,97,80,64,84,73,53,7,34,52,42,34,35,56,44
5_Thursday,1,0,8,32,78,143,58,37,39,93,157,133,69,17,11,6,4,5,7
6_Friday,0,0,0,13,22,19,22,14,8,2,0,0,0,0,0,0,0,0,0
7_Saturday,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,1,3,0,3


In [62]:
# wait-time ribbons were created with R ggplot2 software
# Python packages ggplot or rpy2 could be used for plotting

# select Wednesdays in February for the queueing model
wednesdays = wait_time_120[call_center_data['day_of_week'] == 'Wednesday'] 
wednesdays.head()

C:\Users\graha\AppData\Local\Temp\ipykernel_27584\3379025234.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  wednesdays = wait_time_120[call_center_data['day_of_week'] == 'Wednesday']


,vru+line,call_id,customer_id,priority,type,date,vru_entry,vru_exit,vru_time,q_start,q_exit,q_time,outcome,ser_start,ser_exit,ser_time,server,wait_time,day_of_week,call_hour
163,AA0101,34703,0.0,0,NW,1999-02-03,1900-01-01 08:36:44,8:36:53,9,8:36:53,8:40:08,195,AGENT,8:40:07,8:40:17,10,YIFAT,204,Wednesday,8
171,AA0101,34711,0.0,0,NW,1999-02-03,1900-01-01 10:07:04,10:07:13,9,10:07:13,10:09:39,146,HANG,0:00:00,0:00:00,0,NO_SERVER,155,Wednesday,10
181,AA0101,34722,0.0,0,NW,1999-02-03,1900-01-01 11:46:29,11:46:39,10,11:46:39,11:50:05,206,AGENT,11:50:04,11:51:06,62,YIFAT,216,Wednesday,11
183,AA0101,34724,56051360.0,2,PS,1999-02-03,1900-01-01 12:12:08,12:12:14,6,12:12:14,12:14:43,149,HANG,0:00:00,0:00:00,0,NO_SERVER,155,Wednesday,12
207,AA0101,34748,0.0,0,PS,1999-02-03,1900-01-01 13:31:19,13:31:29,10,13:31:29,13:33:27,118,AGENT,13:33:27,13:37:26,239,KAZAV,128,Wednesday,13


In [63]:
# arrival rate as average number of calls into VRU per hour 
arrived_for_hour = wednesdays['call_hour'].value_counts()
check_hourly_arrival_rate = arrived_for_hour/4  # four Wednesdays in February 1999
check_hourly_arrival_rate

10    24.25
13    21.00
11    20.00
14    18.25
9     18.00
8     16.50
12    16.00
22    14.00
15    13.25
18    13.00
23    11.00
19    10.50
21     8.75
20     8.50
17     8.50
7      5.25
16     1.75
Name: call_hour, dtype: float64

In [64]:
# organize hourly arrival rates according to 24-hour clock
hourly_arrival_rate = [6.75, 1.75, 1.25, 0.00, 0.50, 0.25,\
    4.75, 39.50, 97.25,107.50, 124.00,110.25, 95.50,\
    203.50, 115.75, 115.50, 67.75, 75.00, 88.75,\
    85.50, 68.00, 61.50, 57.50, 44.25]

# service times may vary hour-by-hour due to differences 
# in service requests and individuals calling hour-by-hour
# begin by selecting calls that receive service
wednesdays_served = wednesdays[wednesdays['server'] != \
    'NO_SERVER'] 
wednesdays_served.head() 

,vru+line,call_id,customer_id,priority,type,date,vru_entry,vru_exit,vru_time,q_start,q_exit,q_time,outcome,ser_start,ser_exit,ser_time,server,wait_time,day_of_week,call_hour
163,AA0101,34703,0.0,0,NW,1999-02-03,1900-01-01 08:36:44,8:36:53,9,8:36:53,8:40:08,195,AGENT,8:40:07,8:40:17,10,YIFAT,204,Wednesday,8
181,AA0101,34722,0.0,0,NW,1999-02-03,1900-01-01 11:46:29,11:46:39,10,11:46:39,11:50:05,206,AGENT,11:50:04,11:51:06,62,YIFAT,216,Wednesday,11
207,AA0101,34748,0.0,0,PS,1999-02-03,1900-01-01 13:31:19,13:31:29,10,13:31:29,13:33:27,118,AGENT,13:33:27,13:37:26,239,KAZAV,128,Wednesday,13
209,AA0101,34750,56373913.0,2,PS,1999-02-03,1900-01-01 13:51:19,13:51:25,6,13:51:25,13:53:21,116,AGENT,13:53:20,13:54:35,75,KAZAV,122,Wednesday,13
211,AA0101,34752,0.0,0,NW,1999-02-03,1900-01-01 14:08:10,14:08:19,9,14:08:19,14:14:18,359,AGENT,14:14:17,14:16:44,147,IDIT,368,Wednesday,14


In [65]:
hourly_mean_service_time =\
    wednesdays_served.pivot_table('ser_time', columns = ['call_hour'],\
    aggfunc = 'mean', margins = False)

# hourly service rate given the current numbers of service operators
served_for_hour = wednesdays_served['call_hour'].value_counts()
served_for_hour

10    83
11    68
9     62
13    61
14    55
15    48
22    46
12    46
8     45
18    42
23    37
19    34
17    31
20    30
21    26
7     16
16     6
Name: call_hour, dtype: int64

In [66]:
# compute service rate noting that there are 3600 seconds in an hour
# adding 60 seconds to each mean service time for time between calls
# this 60 seconds is the wrap up time or time an service agent remains 
# unavailable to answer a new call after a call has been completed
mean_hourly_service_rate = 3600/(hourly_mean_service_time.mean() + 60)
round(mean_hourly_service_rate,3)

call_hour
7     21.711
8     16.202
9     13.320
10    17.388
11    13.879
12    13.889
13    15.541
14    17.691
15    15.023
16    25.899
17    16.930
18    22.144
19    17.143
20    15.511
21    17.965
22    18.067
23    16.613
dtype: float64

In [67]:
# use 15 calls per hour as the rate for one service operator
SERVICE_RATE = 15

# use a target for the probability of waiting in queue to be 0.50
PROBABILITY_GOAL = 0.50

# Erlang C queueing calculations with Python erlang_C function
# inputs c = number of servers
#        r = ratio of rate of arrivals and rate of service
# returns the propability of waiting in queue because all servers are busy
# use while-loop iteration to determine the number of servers needed 
# we do this for each hour of the day knowing the hourly arrival rate
servers_needed = [0] * 24
for index_for_hour in range(24):
    if (hourly_arrival_rate[index_for_hour] > 0):
        erlang_probability = 1 # initialize on entering while-loop
        while (erlang_probability > PROBABILITY_GOAL):
            servers_needed[index_for_hour] = servers_needed[index_for_hour] + 1
            erlang_probability = \
                erlang_C(c = servers_needed[index_for_hour],\
                    r = hourly_arrival_rate[index_for_hour]/SERVICE_RATE)
print(servers_needed)  # check queueing theory result 
# the result for servers.needed is obtained as
# 1  1  1  0  1  1  1  4  8  9 10  9  8 16 10 10  6  7  8  8  6  6  5  4
# we will assume the bank call center will be closed hours 00 through 05
# but use the other values as the bank's needed numbers of servers
for index_for_hour in range(6):
    servers_needed[index_for_hour] = 0
print('\nHourly Operator Requirements:\n',servers_needed)

[1, 1, 1, 0, 1, 1, 1, 4, 8, 9, 10, 9, 8, 16, 10, 10, 6, 7, 8, 8, 6, 6, 5, 4]

Hourly Operator Requirements:
 [0, 0, 0, 0, 0, 0, 1, 4, 8, 9, 10, 9, 8, 16, 10, 10, 6, 7, 8, 8, 6, 6, 5, 4]


In [68]:
# read in case data for the structure of call center worker shifts
bank_shifts_data_frame = pd.read_csv("data_anonymous_bank_shifts.csv")
# examine the structure of these data
bank_shifts_data_frame.head()

,Hour,StartTime,Shift1,Shift2,Shift3,Shift4,Shift5,Shift6,Shift7,Shift8
0,1,Midnight,1,0,0,0,0,0,0,0
1,2,1am,1,0,0,0,0,0,0,0
2,3,2am,1,0,0,0,0,0,0,0
3,4,3am,1,0,0,0,0,0,0,0
4,5,4am,1,0,0,0,0,0,0,0


In [69]:
# constraint matrix as required for mathematical programming
constraint_matrix = np.array(bank_shifts_data_frame)[:,2:]
# we will create this type of object on the R side as well

# six-hour shift salaries in Israeli sheqels 
# 1 ILS = 3.61 USD in June 2013
# these go into the objective function for integer programing
# with the objective of minimizing total costs
cost_vector = [252, 288, 180, 180, 180, 288, 288, 288] 

# install lpsolove package and drivers for Python 
# noting the operating system being used
# or use rpy2 access to lpSolve in R as shown here

# assign lists from Python to R using rpy2
r.assign('servers_needed_R', servers_needed)
r.assign('cost_vector_R', cost_vector)

r('bank.shifts.data.frame <- read.csv("data_anonymous_bank_shifts.csv")')
r('constraint_matrix_R <- as.matrix(bank.shifts.data.frame[,3:10])')

# check mathematical programming inputs on the R side
r('print(as.numeric(unlist(servers_needed_R)))')

 [1]  0  0  0  0  0  0  1  4  8  9 10  9  8 16 10 10  6  7  8  8  6  6  5  4


0.000000,0.000000,0.000000,...,6.000000,5.000000,4.000000


In [70]:
r('print(as.numeric(unlist(cost_vector_R)))')

[1] 252 288 180 180 180 288 288 288


252.000000,288.000000,180.000000,...,288.000000,288.000000,288.000000


In [71]:
r('print(constraint_matrix_R)')

      Shift1 Shift2 Shift3 Shift4 Shift5 Shift6 Shift7 Shift8
 [1,]      1      0      0      0      0      0      0      0
 [2,]      1      0      0      0      0      0      0      0
 [3,]      1      0      0      0      0      0      0      0
 [4,]      1      0      0      0      0      0      0      0
 [5,]      1      0      0      0      0      0      0      0
 [6,]      1      0      0      0      0      0      0      0
 [7,]      0      1      0      0      0      0      0      0
 [8,]      0      1      0      0      0      0      0      0
 [9,]      0      1      1      0      0      0      0      0
[10,]      0      1      1      0      0      0      0      0
[11,]      0      1      1      1      0      0      0      0
[12,]      0      1      1      1      0      0      0      0
[13,]      0      0      1      1      1      0      0      0
[14,]      0      0      1      1      1      0      0      0
[15,]      0      0      0      1      1      1      0      0
[16,]   

1,1,1,...,1,1,1


In [72]:
# solve the mathematical programming problem
r('library(lpSolve)')  
r('call_center_schedule <- lp(const.mat=constraint_matrix_R,\
    const.rhs = as.numeric(unlist(servers_needed_R)),\
    const.dir = rep(">=", times = 8),\
    int.vec = 1:8,\
    objective = as.numeric(unlist(cost_vector_R)),\
    direction = "min")')
    
# prepare summary of the results for the call center problem
# working on the R side
r('ShiftID <- 1:8')
r('StartTime <- c(0,6,8,10,12,2,4,6)')
# c("Midnight","6 AM","8 AM","10 AM","Noon","2 PM","4 PM","6 PM")
r('ShiftDuration <- rep(6,times=8)')
r('HourlyShiftSalary <- c(42,48,30,30,30,48,48,48)')
r('HourlyShiftCost <- call_center_schedule$objective') # six x hourly shift salary
r('Solution <- call_center_schedule$solution')  
r('ShiftCost <- call_center_schedule$solution * call_center_schedule$objective')
r('call_center_summary <- \
  data.frame(ShiftID,StartTime,ShiftDuration,HourlyShiftSalary,\
  HourlyShiftCost,Solution,ShiftCost)')
r('cat("\n\n","Call Center Summary","\n\n")')
r('print(call_center_summary)')



 Call Center Summary 

  ShiftID StartTime ShiftDuration HourlyShiftSalary HourlyShiftCost Solution
1       1         0             6                42             252        0
2       2         6             6                48             288        4
3       3         8             6                30             180        8
4       4        10             6                30             180        4
5       5        12             6                30             180        4
6       6         2             6                48             288        2
7       7         4             6                48             288        1
8       8         6             6                48             288        5
  ShiftCost
1         0
2      1152
3      1440
4       720
5       720
6       576
7       288
8      1440


ShiftID,StartTime,ShiftDuration,HourlyShiftSalary,HourlyShiftCost,Solution,ShiftCost
...,...,...,...,...,...,...


In [73]:
r('print(call_center_schedule)')

# alternatively... bring the solution from R to Python
# and print the minimum-cost solution on the Python side
call_center_schedule = r('call_center_schedule')
print(call_center_schedule)

# Suggestion for the student:
# Attack the problem using discrete event simulation, 
# perhaps drawing on the SimPy package.
# Try running a sensitivity test, varying the workforce requirements
# and noting the effect upon the optimal assignment of workers to shifts.
# This can be done in a Python for-loop.

Success: the objective function is 6336 
Success: the objective function is 6336 

